In [12]:
import pandas as pd

art_gallerys = pd.read_csv("D:/DataSets/Services/Filtered_art_gallerys.csv")

In [13]:
art_gallerys['geoLocation'] = art_gallerys['geometry'].str.replace(',', ' ', regex=True)
art_gallerys['neighbourhoodGeoLocation'] = art_gallerys['neighborhood'].str.replace(',', ' ', regex=True)

In [14]:
art_gallerys = art_gallerys.drop("geometry", axis = 1)
art_gallerys = art_gallerys.drop("neighborhood", axis = 1)

In [16]:
# Split the geoLocation column into latitude and longitude columns
art_gallerys[['latitude', 'longitude']] = art_gallerys['geoLocation'].str.split(' ', expand=True)

# Format the data into the expected SQL format
art_gallerys['geometry'] = art_gallerys.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
art_gallerys = art_gallerys.drop(columns=['geoLocation', 'latitude', 'longitude'])

In [18]:
# Split the geoLocation column into latitude and longitude columns
art_gallerys[['latitude', 'longitude']] = art_gallerys['neighbourhoodGeoLocation'].str.split(' ', expand=True)

# Format the data into the expected SQL format
art_gallerys['neighbourhoodGeometry'] = art_gallerys.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
art_gallerys = art_gallerys.drop(columns=['neighbourhoodGeoLocation', 'latitude', 'longitude'])

In [22]:
art_gallerys.to_csv("D:/DataSets/ServicesFinished/art_gallerys.csv", index = False)

# NEIGHBOURHOODS

In [91]:
# create table that contains unique neighbourhood names
addresses = pd.read_csv("D:/DataSets/backup_finished/addresses.csv")

In [106]:
unique_neighborhoods = addresses['neighbourhoodGeoLocation'].unique()

# Create a new DataFrame with a column for unique neighborhood names
neighbour_input = pd.DataFrame({'neighbourhoodGeoLocation': unique_neighborhoods})

In [107]:
neighbour_input['neighbourhoodGeoLocation'] = neighbour_input['neighbourhoodGeoLocation'].str.replace(r'POINT\(([^ ]+) ([^)]+)\)', r'\1 \2')

C:\Users\seanl\AppData\Local\Temp\ipykernel_24652\4112417616.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  neighbour_input['neighbourhoodGeoLocation'] = neighbour_input['neighbourhoodGeoLocation'].str.replace(r'POINT\(([^ ]+) ([^)]+)\)', r'\1 \2')


In [108]:
neighbour_input['neighbourhoodGeoLocation'] = neighbour_input['neighbourhoodGeoLocation'].str.replace(' ', ',')

In [231]:
neighborhood_df

,neighbourhoodGeoLocation
0,"39.36253019999999,-76.6595727"
1,"39.2842097,-76.593611"
2,"39.3226705,-76.5846633"
3,"39.292007,-76.6167212"
4,"39.3481652,-76.6831397"
...,...
76,"39.30592439999999,-76.6650946"
77,"39.3041275,-76.6441591"
78,"39.3625548,-76.544488"
79,"39.2208578,-76.5863882"


In [39]:
neighborhood_df.insert(0,"neighbourhoodID", neighborhood_df.pop("neighbourhoodID"))

In [36]:
# Get unique 'neighbourhoodGeoLocation' values from both DataFrames
neighbourhood_geo_locations_neighbourhoods = neighborhood_df['UniqueNeighborhood'].unique()
neighbourhood_geo_locations_artgallerys = art_gallerys['neighbourhoodGeoLocation'].unique()

# Check if all 'neighbourhoodGeoLocation' values from 'neighbourhoods' exist in 'artgallerys'
all_exist = all(geo_location in neighbourhood_geo_locations_artgallerys for geo_location in neighbourhood_geo_locations_neighbourhoods)

if all_exist:
    print("All neighbourhoodGeoLocation values from neighbourhoods exist in artgallerys.")
else:
    print("Some neighbourhoodGeoLocation values from neighbourhoods do not exist in artgallerys.")

Some neighbourhoodGeoLocation values from neighbourhoods do not exist in artgallerys.


In [38]:
neighborhood_df['neighbourhoodID'] = range(len(neighborhood_df))

In [41]:
neighborhood_df.to_csv("D:/DataSets/ServicesFinished/neighbourhoods.csv", index = False)

# PULL API KEY

In [266]:
api_path = "C:/Users/seanl/googlemaps_api.txt" 

with open(api_path, "r") as file:
    api_file = file.read()

# Define API function

In [293]:
import googlemaps
import pandas as pd
import time

# DEFINE BUSINESS TYPE HERE: IT'S ALL YOU NEED TO CHANGE:

# Initialize Google Maps client with your API key

map_client = googlemaps.Client(api_file)

# Define a function to fetch nightclubs
def fetch_businesses(input_neighbourhood, business):
    allowed_columns = ["place_id", "geometry"]

    df_list = []

    # Perform the initial request
    response = map_client.places_nearby(
        location=input_neighbourhood,
        radius=1500,
        type= business,
    )

    while 'results' in response:
        for result in response['results']:
            filtered_result = {key: result[key] for key in allowed_columns}
            filtered_result['input_neighbourhood_geoLocation'] = input_neighbourhood

            df_list.append(filtered_result)

        time.sleep(2)  # Pausing to avoid rate limiting issues
        if 'next_page_token' in response:
            response = map_client.places_nearby(
                location=input_neighbourhood,
                radius=1500,
                type= business,
                page_token=response['next_page_token']
            )
        else:
            break

    return df_list

 # FILTERED BANKS

In [282]:
banks = pd.read_csv("D:/DataSets/Services/Filtered_banks.csv")

In [283]:
import googlemaps
import pandas as pd
import time

# DEFINE BUSINESS TYPE HERE: IT'S ALL YOU NEED TO CHANGE:

# Initialize Google Maps client with your API key

map_client = googlemaps.Client(api_file)

# Define a function to fetch nightclubs
def fetch_businesses(input_neighbourhood):
    allowed_columns = ["place_id", "geometry"]

    df_list = []

    # Perform the initial request
    response = map_client.places_nearby(
        location=input_neighbourhood,
        radius=1500,
        type= "bank",
    )

    while 'results' in response:
        for result in response['results']:
            filtered_result = {key: result[key] for key in allowed_columns}
            filtered_result['input_neighbourhood_geoLocation'] = input_neighbourhood

            df_list.append(filtered_result)

        time.sleep(2)  # Pausing to avoid rate limiting issues
        if 'next_page_token' in response:
            response = map_client.places_nearby(
                location=input_neighbourhood,
                radius=1500,
                type= "banks",
                page_token=response['next_page_token']
            )
        else:
            break

    return df_list

#USE FUNCTION
# Create a list to store all nightclubs
all_businesses_clubs = []

for index, row in neighborhood_df.iterrows():
    input_neighbourhood = row["neighbourhoodGeoLocation"]
    neighborhood_businesses = fetch_businesses(input_neighbourhood)
    all_businesses_clubs.extend(neighborhood_businesses)

# Create a DataFrame from the combined list of nightclubs
banks = pd.DataFrame(all_businesses_clubs)

banks['geometry'] = banks['geometry'].astype(str)

# fix the geometry string
for i in range(len(banks)):
    input_str = banks["geometry"][i]
    data_dict = ast.literal_eval(input_str)
    lat = data_dict['location']['lat']
    lng = data_dict['location']['lng']
    result_str = f"{lat} {lng}"
    banks["geometry"][i] = result_str
    
#add point to geometry
# Split the geoLocation column into latitude and longitude columns
banks[['latitude', 'longitude']] = banks['geometry'].str.split(' ', expand=True)

# Format the data into the expected SQL format
banks['geoLocation'] = banks.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
banks = banks.drop(columns=['geometry', 'latitude', 'longitude'])

#add point to neighbourhoods
# Split the geoLocation column into latitude and longitude columns
banks[['latitude', 'longitude']] = banks['input_neighbourhood_geoLocation'].str.split(',', expand=True)

# Format the data into the expected SQL format
banks['neighbourhoodInput'] = banks.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
banks = banks.drop(columns=['input_neighbourhood_geoLocation', 'latitude', 'longitude'])

In [285]:
banks.to_csv("D:/DataSets/NewServices/banks.csv", index = False)

# FILTERED BARS

In [289]:
bars = pd.read_csv("D:/DataSets/Services/Filtered_bars.csv")

In [290]:
# Create a list to store all nightclubs
all_businesses_clubs = []

for index, row in neighborhood_df.iterrows():
    input_neighbourhood = row["neighbourhoodGeoLocation"]
    neighborhood_businesses = fetch_businesses(input_neighbourhood,"bar")
    all_businesses_clubs.extend(neighborhood_businesses)

# Create a DataFrame from the combined list of nightclubs
bars = pd.DataFrame(all_businesses_clubs)

bars['geometry'] = bars['geometry'].astype(str)

# fix the geometry string
for i in range(len(bars)):
    input_str = bars["geometry"][i]
    data_dict = ast.literal_eval(input_str)
    lat = data_dict['location']['lat']
    lng = data_dict['location']['lng']
    result_str = f"{lat} {lng}"
    bars["geometry"][i] = result_str
    
#add point to geometry
# Split the geoLocation column into latitude and longitude columns
bars[['latitude', 'longitude']] = bars['geometry'].str.split(' ', expand=True)

# Format the data into the expected SQL format
bars['geoLocation'] = bars.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
bars = bars.drop(columns=['geometry', 'latitude', 'longitude'])

#add point to neighbourhoods
# Split the geoLocation column into latitude and longitude columns
bars[['latitude', 'longitude']] = bars['input_neighbourhood_geoLocation'].str.split(',', expand=True)

# Format the data into the expected SQL format
bars['neighbourhoodInput'] = bars.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
bars = bars.drop(columns=['input_neighbourhood_geoLocation', 'latitude', 'longitude'])

bars = bars.rename(columns = {"neighbourhoodInput" : "neighbourhoodGeoLocation"})

In [296]:
bars.to_csv("D:/DataSets/NewServices//bars.csv", index = False)

# FILTERED BEAUTY SALONS

In [298]:
beauty = pd.read_csv("D:/DataSets/Services/Filtered_bars.csv")
df = beauty.copy()

In [299]:
# Create a list to store all nightclubs
all_businesses_clubs = []

for index, row in neighborhood_df.iterrows():
    input_neighbourhood = row["neighbourhoodGeoLocation"]
    neighborhood_businesses = fetch_businesses(input_neighbourhood,"beauty_salon")
    all_businesses_clubs.extend(neighborhood_businesses)

# Create a DataFrame from the combined list of nightclubs
df = pd.DataFrame(all_businesses_clubs)

df['geometry'] = df['geometry'].astype(str)

# fix the geometry string
for i in range(len(df)):
    input_str = df["geometry"][i]
    data_dict = ast.literal_eval(input_str)
    lat = data_dict['location']['lat']
    lng = data_dict['location']['lng']
    result_str = f"{lat} {lng}"
    df["geometry"][i] = result_str
    
#add point to geometry
# Split the geoLocation column into latitude and longitude columns
df[['latitude', 'longitude']] = df['geometry'].str.split(' ', expand=True)

# Format the data into the expected SQL format
df['geoLocation'] = df.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
df = df.drop(columns=['geometry', 'latitude', 'longitude'])

#add point to neighbourhoods
# Split the geoLocation column into latitude and longitude columns
df[['latitude', 'longitude']] = df['input_neighbourhood_geoLocation'].str.split(',', expand=True)

# Format the data into the expected SQL format
df['neighbourhoodInput'] = df.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
df = df.drop(columns=['input_neighbourhood_geoLocation', 'latitude', 'longitude'])

df = df.rename(columns = {"neighbourhoodInput" : "neighbourhoodGeoLocation"})

In [300]:
beauty = df.copy()

In [303]:
beauty.to_csv("D:/DataSets/NewServices//beauty_salons.csv", index = False)

# FILTERED BUS STATIONS

In [304]:
bus_stations = pd.read_csv("D:/DataSets/Services/Filtered_bus_station.csv")
df = bus_stations.copy()

In [305]:
# Create a list to store all nightclubs
all_businesses_clubs = []

for index, row in neighborhood_df.iterrows():
    input_neighbourhood = row["neighbourhoodGeoLocation"]
    neighborhood_businesses = fetch_businesses(input_neighbourhood,"bus_station")
    all_businesses_clubs.extend(neighborhood_businesses)

# Create a DataFrame from the combined list of nightclubs
df = pd.DataFrame(all_businesses_clubs)

df['geometry'] = df['geometry'].astype(str)

# fix the geometry string
for i in range(len(df)):
    input_str = df["geometry"][i]
    data_dict = ast.literal_eval(input_str)
    lat = data_dict['location']['lat']
    lng = data_dict['location']['lng']
    result_str = f"{lat} {lng}"
    df["geometry"][i] = result_str
    
#add point to geometry
# Split the geoLocation column into latitude and longitude columns
df[['latitude', 'longitude']] = df['geometry'].str.split(' ', expand=True)

# Format the data into the expected SQL format
df['geoLocation'] = df.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
df = df.drop(columns=['geometry', 'latitude', 'longitude'])

#add point to neighbourhoods
# Split the geoLocation column into latitude and longitude columns
df[['latitude', 'longitude']] = df['input_neighbourhood_geoLocation'].str.split(',', expand=True)

# Format the data into the expected SQL format
df['neighbourhoodInput'] = df.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
df = df.drop(columns=['input_neighbourhood_geoLocation', 'latitude', 'longitude'])

df = df.rename(columns = {"neighbourhoodInput" : "neighbourhoodGeoLocation"})

In [306]:
df.to_csv("D:/DataSets/NewServices/bus_stations.csv", index = False)

# FILTERED CAFES

In [312]:
cafes = pd.read_csv("D:/DataSets/Services/Filtered_cafe.csv")
df = cafes.copy()

In [313]:
# Create a list to store all nightclubs
all_businesses_clubs = []

for index, row in neighborhood_df.iterrows():
    input_neighbourhood = row["neighbourhoodGeoLocation"]
    neighborhood_businesses = fetch_businesses(input_neighbourhood,"cafe")
    all_businesses_clubs.extend(neighborhood_businesses)

# Create a DataFrame from the combined list of nightclubs
df = pd.DataFrame(all_businesses_clubs)

df['geometry'] = df['geometry'].astype(str)

# fix the geometry string
for i in range(len(df)):
    input_str = df["geometry"][i]
    data_dict = ast.literal_eval(input_str)
    lat = data_dict['location']['lat']
    lng = data_dict['location']['lng']
    result_str = f"{lat} {lng}"
    df["geometry"][i] = result_str
    
#add point to geometry
# Split the geoLocation column into latitude and longitude columns
df[['latitude', 'longitude']] = df['geometry'].str.split(' ', expand=True)

# Format the data into the expected SQL format
df['geoLocation'] = df.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
df = df.drop(columns=['geometry', 'latitude', 'longitude'])

#add point to neighbourhoods
# Split the geoLocation column into latitude and longitude columns
df[['latitude', 'longitude']] = df['input_neighbourhood_geoLocation'].str.split(',', expand=True)

# Format the data into the expected SQL format
df['neighbourhoodInput'] = df.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
df = df.drop(columns=['input_neighbourhood_geoLocation', 'latitude', 'longitude'])

df = df.rename(columns = {"neighbourhoodInput" : "neighbourhoodGeoLocation"})

In [314]:
df.to_csv("D:/DataSets/NewServices/cafes.csv", index = False)

# FILTERED FIRE STATION

In [315]:
fire_stations = pd.read_csv("D:/DataSets/Services/Filtered_fire_station.csv")
df = fire_stations.copy()


In [316]:
# Create a list to store all nightclubs
all_businesses_clubs = []

for index, row in neighborhood_df.iterrows():
    input_neighbourhood = row["neighbourhoodGeoLocation"]
    neighborhood_businesses = fetch_businesses(input_neighbourhood,"fire_station")
    all_businesses_clubs.extend(neighborhood_businesses)

# Create a DataFrame from the combined list of nightclubs
df = pd.DataFrame(all_businesses_clubs)

df['geometry'] = df['geometry'].astype(str)

# fix the geometry string
for i in range(len(df)):
    input_str = df["geometry"][i]
    data_dict = ast.literal_eval(input_str)
    lat = data_dict['location']['lat']
    lng = data_dict['location']['lng']
    result_str = f"{lat} {lng}"
    df["geometry"][i] = result_str
    
#add point to geometry
# Split the geoLocation column into latitude and longitude columns
df[['latitude', 'longitude']] = df['geometry'].str.split(' ', expand=True)

# Format the data into the expected SQL format
df['geoLocation'] = df.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
df = df.drop(columns=['geometry', 'latitude', 'longitude'])

#add point to neighbourhoods
# Split the geoLocation column into latitude and longitude columns
df[['latitude', 'longitude']] = df['input_neighbourhood_geoLocation'].str.split(',', expand=True)

# Format the data into the expected SQL format
df['neighbourhoodInput'] = df.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
df = df.drop(columns=['input_neighbourhood_geoLocation', 'latitude', 'longitude'])

df = df.rename(columns = {"neighbourhoodInput" : "neighbourhoodGeoLocation"})

In [317]:
df.to_csv("D:/DataSets/NewServices/fire_stations.csv", index = False)

# FILTERED GYMS

In [318]:
gyms = pd.read_csv("D:/DataSets/Services/Filtered_gyms.csv")
df = gyms.copy()


In [319]:
# Create a list to store all nightclubs
all_businesses_clubs = []

for index, row in neighborhood_df.iterrows():
    input_neighbourhood = row["neighbourhoodGeoLocation"]
    neighborhood_businesses = fetch_businesses(input_neighbourhood,"gym")
    all_businesses_clubs.extend(neighborhood_businesses)

# Create a DataFrame from the combined list of nightclubs
df = pd.DataFrame(all_businesses_clubs)

df['geometry'] = df['geometry'].astype(str)

# fix the geometry string
for i in range(len(df)):
    input_str = df["geometry"][i]
    data_dict = ast.literal_eval(input_str)
    lat = data_dict['location']['lat']
    lng = data_dict['location']['lng']
    result_str = f"{lat} {lng}"
    df["geometry"][i] = result_str
    
#add point to geometry
# Split the geoLocation column into latitude and longitude columns
df[['latitude', 'longitude']] = df['geometry'].str.split(' ', expand=True)

# Format the data into the expected SQL format
df['geoLocation'] = df.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
df = df.drop(columns=['geometry', 'latitude', 'longitude'])

#add point to neighbourhoods
# Split the geoLocation column into latitude and longitude columns
df[['latitude', 'longitude']] = df['input_neighbourhood_geoLocation'].str.split(',', expand=True)

# Format the data into the expected SQL format
df['neighbourhoodInput'] = df.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
df = df.drop(columns=['input_neighbourhood_geoLocation', 'latitude', 'longitude'])

df = df.rename(columns = {"neighbourhoodInput" : "neighbourhoodGeoLocation"})

In [320]:
df.to_csv("D:/DataSets/NewServices/gyms.csv", index = False)

# FILTERED HOSPITALS

In [321]:
hospitals = pd.read_csv("D:/DataSets/Services/Filtered_hospitals.csv")
df = hospitals.copy()


In [322]:
# Create a list to store all nightclubs
all_businesses_clubs = []

for index, row in neighborhood_df.iterrows():
    input_neighbourhood = row["neighbourhoodGeoLocation"]
    neighborhood_businesses = fetch_businesses(input_neighbourhood,"hospital")
    all_businesses_clubs.extend(neighborhood_businesses)

# Create a DataFrame from the combined list of nightclubs
df = pd.DataFrame(all_businesses_clubs)

df['geometry'] = df['geometry'].astype(str)

# fix the geometry string
for i in range(len(df)):
    input_str = df["geometry"][i]
    data_dict = ast.literal_eval(input_str)
    lat = data_dict['location']['lat']
    lng = data_dict['location']['lng']
    result_str = f"{lat} {lng}"
    df["geometry"][i] = result_str
    
#add point to geometry
# Split the geoLocation column into latitude and longitude columns
df[['latitude', 'longitude']] = df['geometry'].str.split(' ', expand=True)

# Format the data into the expected SQL format
df['geoLocation'] = df.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
df = df.drop(columns=['geometry', 'latitude', 'longitude'])

#add point to neighbourhoods
# Split the geoLocation column into latitude and longitude columns
df[['latitude', 'longitude']] = df['input_neighbourhood_geoLocation'].str.split(',', expand=True)

# Format the data into the expected SQL format
df['neighbourhoodInput'] = df.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
df = df.drop(columns=['input_neighbourhood_geoLocation', 'latitude', 'longitude'])

df = df.rename(columns = {"neighbourhoodInput" : "neighbourhoodGeoLocation"})

In [323]:
df.to_csv("D:/DataSets/NewServices/hospitals.csv", index = False)

# FILTERED NIGHT CLUBS

In [324]:
night_clubs = pd.read_csv("D:/DataSets/Services/Filtered_night_clubs.csv")
df = night_clubs.copy()


In [325]:
# Create a list to store all nightclubs
all_businesses_clubs = []

for index, row in neighborhood_df.iterrows():
    input_neighbourhood = row["neighbourhoodGeoLocation"]
    neighborhood_businesses = fetch_businesses(input_neighbourhood,"night_club")
    all_businesses_clubs.extend(neighborhood_businesses)

# Create a DataFrame from the combined list of nightclubs
df = pd.DataFrame(all_businesses_clubs)

df['geometry'] = df['geometry'].astype(str)

# fix the geometry string
for i in range(len(df)):
    input_str = df["geometry"][i]
    data_dict = ast.literal_eval(input_str)
    lat = data_dict['location']['lat']
    lng = data_dict['location']['lng']
    result_str = f"{lat} {lng}"
    df["geometry"][i] = result_str
    
#add point to geometry
# Split the geoLocation column into latitude and longitude columns
df[['latitude', 'longitude']] = df['geometry'].str.split(' ', expand=True)

# Format the data into the expected SQL format
df['geoLocation'] = df.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
df = df.drop(columns=['geometry', 'latitude', 'longitude'])

#add point to neighbourhoods
# Split the geoLocation column into latitude and longitude columns
df[['latitude', 'longitude']] = df['input_neighbourhood_geoLocation'].str.split(',', expand=True)

# Format the data into the expected SQL format
df['neighbourhoodInput'] = df.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
df = df.drop(columns=['input_neighbourhood_geoLocation', 'latitude', 'longitude'])

df = df.rename(columns = {"neighbourhoodInput" : "neighbourhoodGeoLocation"})

In [326]:
df.to_csv("D:/DataSets/NewServices/night_clubs.csv", index = False)

# FILTERED PARKS

In [327]:
filtered_parks = pd.read_csv("D:/DataSets/Services/Filtered_parks.csv")
df = filtered_parks.copy()


In [ ]:
# Create a list to store all nightclubs
all_businesses_clubs = []

for index, row in neighborhood_df.iterrows():
    input_neighbourhood = row["neighbourhoodGeoLocation"]
    neighborhood_businesses = fetch_businesses(input_neighbourhood,"park")
    all_businesses_clubs.extend(neighborhood_businesses)

# Create a DataFrame from the combined list of nightclubs
df = pd.DataFrame(all_businesses_clubs)

df['geometry'] = df['geometry'].astype(str)

# fix the geometry string
for i in range(len(df)):
    input_str = df["geometry"][i]
    data_dict = ast.literal_eval(input_str)
    lat = data_dict['location']['lat']
    lng = data_dict['location']['lng']
    result_str = f"{lat} {lng}"
    df["geometry"][i] = result_str
    
#add point to geometry
# Split the geoLocation column into latitude and longitude columns
df[['latitude', 'longitude']] = df['geometry'].str.split(' ', expand=True)

# Format the data into the expected SQL format
df['geoLocation'] = df.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
df = df.drop(columns=['geometry', 'latitude', 'longitude'])

#add point to neighbourhoods
# Split the geoLocation column into latitude and longitude columns
df[['latitude', 'longitude']] = df['input_neighbourhood_geoLocation'].str.split(',', expand=True)

# Format the data into the expected SQL format
df['neighbourhoodInput'] = df.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
df = df.drop(columns=['input_neighbourhood_geoLocation', 'latitude', 'longitude'])

df = df.rename(columns = {"neighbourhoodInput" : "neighbourhoodGeoLocation"})

In [ ]:
df.to_csv("D:/DataSets/NewServices/parks.csv", index = False)

# FILTERED PHARMACIES

In [84]:
filtered_pharmacies = pd.read_csv("D:/DataSets/Services/Filtered_pharmacy.csv")
df =filtered_pharmacies.copy()


In [ ]:
# Create a list to store all nightclubs
all_businesses_clubs = []

for index, row in neighborhood_df.iterrows():
    input_neighbourhood = row["neighbourhoodGeoLocation"]
    neighborhood_businesses = fetch_businesses(input_neighbourhood,"pharmacy")
    all_businesses_clubs.extend(neighborhood_businesses)

# Create a DataFrame from the combined list of nightclubs
df = pd.DataFrame(all_businesses_clubs)

df['geometry'] = df['geometry'].astype(str)

# fix the geometry string
for i in range(len(df)):
    input_str = df["geometry"][i]
    data_dict = ast.literal_eval(input_str)
    lat = data_dict['location']['lat']
    lng = data_dict['location']['lng']
    result_str = f"{lat} {lng}"
    df["geometry"][i] = result_str
    
#add point to geometry
# Split the geoLocation column into latitude and longitude columns
df[['latitude', 'longitude']] = df['geometry'].str.split(' ', expand=True)

# Format the data into the expected SQL format
df['geoLocation'] = df.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
df = df.drop(columns=['geometry', 'latitude', 'longitude'])

#add point to neighbourhoods
# Split the geoLocation column into latitude and longitude columns
df[['latitude', 'longitude']] = df['input_neighbourhood_geoLocation'].str.split(',', expand=True)

# Format the data into the expected SQL format
df['neighbourhoodInput'] = df.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
df = df.drop(columns=['input_neighbourhood_geoLocation', 'latitude', 'longitude'])

df = df.rename(columns = {"neighbourhoodInput" : "neighbourhoodGeoLocation"})

In [ ]:
df.to_csv("D:/DataSets/NewServices/pharmacies.csv", index = False)

# FILTERED POLICE STATION

In [85]:
filtered_police = pd.read_csv("D:/DataSets/Services/Filtered_police.csv")
df = filtered_police.copy()


In [ ]:
# Create a list to store all nightclubs
all_businesses_clubs = []

for index, row in neighborhood_df.iterrows():
    input_neighbourhood = row["neighbourhoodGeoLocation"]
    neighborhood_businesses = fetch_businesses(input_neighbourhood,"police_station")
    all_businesses_clubs.extend(neighborhood_businesses)

# Create a DataFrame from the combined list of nightclubs
df = pd.DataFrame(all_businesses_clubs)

df['geometry'] = df['geometry'].astype(str)

# fix the geometry string
for i in range(len(df)):
    input_str = df["geometry"][i]
    data_dict = ast.literal_eval(input_str)
    lat = data_dict['location']['lat']
    lng = data_dict['location']['lng']
    result_str = f"{lat} {lng}"
    df["geometry"][i] = result_str
    
#add point to geometry
# Split the geoLocation column into latitude and longitude columns
df[['latitude', 'longitude']] = df['geometry'].str.split(' ', expand=True)

# Format the data into the expected SQL format
df['geoLocation'] = df.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
df = df.drop(columns=['geometry', 'latitude', 'longitude'])

#add point to neighbourhoods
# Split the geoLocation column into latitude and longitude columns
df[['latitude', 'longitude']] = df['input_neighbourhood_geoLocation'].str.split(',', expand=True)

# Format the data into the expected SQL format
df['neighbourhoodInput'] = df.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
df = df.drop(columns=['input_neighbourhood_geoLocation', 'latitude', 'longitude'])

df = df.rename(columns = {"neighbourhoodInput" : "neighbourhoodGeoLocation"})

In [ ]:
df.to_csv("D:/DataSets/NewServices/police_stations.csv", index = False)

# FILTERED RESTUARANTS

In [86]:
filtered_restaurants = pd.read_csv("D:/DataSets/Services/Filtered_restaurant.csv")
df = filtered_restaurant.copy()


In [ ]:
# Create a list to store all nightclubs
all_businesses_clubs = []

for index, row in neighborhood_df.iterrows():
    input_neighbourhood = row["neighbourhoodGeoLocation"]
    neighborhood_businesses = fetch_businesses(input_neighbourhood,"restaurant")
    all_businesses_clubs.extend(neighborhood_businesses)

# Create a DataFrame from the combined list of nightclubs
df = pd.DataFrame(all_businesses_clubs)

df['geometry'] = df['geometry'].astype(str)

# fix the geometry string
for i in range(len(df)):
    input_str = df["geometry"][i]
    data_dict = ast.literal_eval(input_str)
    lat = data_dict['location']['lat']
    lng = data_dict['location']['lng']
    result_str = f"{lat} {lng}"
    df["geometry"][i] = result_str
    
#add point to geometry
# Split the geoLocation column into latitude and longitude columns
df[['latitude', 'longitude']] = df['geometry'].str.split(' ', expand=True)

# Format the data into the expected SQL format
df['geoLocation'] = df.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
df = df.drop(columns=['geometry', 'latitude', 'longitude'])

#add point to neighbourhoods
# Split the geoLocation column into latitude and longitude columns
df[['latitude', 'longitude']] = df['input_neighbourhood_geoLocation'].str.split(',', expand=True)

# Format the data into the expected SQL format
df['neighbourhoodInput'] = df.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
df = df.drop(columns=['input_neighbourhood_geoLocation', 'latitude', 'longitude'])

df = df.rename(columns = {"neighbourhoodInput" : "neighbourhoodGeoLocation"})

In [ ]:
df.to_csv("D:/DataSets/NewServices/restaurants.csv", index = False)

# FILTERED SUPERMARKETS

In [88]:
filtered_supermarkets = pd.read_csv("D:/DataSets/Services/Filtered_supermarketss.csv")
df = filtered_supermarkets.copy()


In [ ]:
# Create a list to store all nightclubs
all_businesses_clubs = []

for index, row in neighborhood_df.iterrows():
    input_neighbourhood = row["neighbourhoodGeoLocation"]
    neighborhood_businesses = fetch_businesses(input_neighbourhood,"supermarket")
    all_businesses_clubs.extend(neighborhood_businesses)

# Create a DataFrame from the combined list of nightclubs
df = pd.DataFrame(all_businesses_clubs)

df['geometry'] = df['geometry'].astype(str)

# fix the geometry string
for i in range(len(df)):
    input_str = df["geometry"][i]
    data_dict = ast.literal_eval(input_str)
    lat = data_dict['location']['lat']
    lng = data_dict['location']['lng']
    result_str = f"{lat} {lng}"
    df["geometry"][i] = result_str
    
#add point to geometry
# Split the geoLocation column into latitude and longitude columns
df[['latitude', 'longitude']] = df['geometry'].str.split(' ', expand=True)

# Format the data into the expected SQL format
df['geoLocation'] = df.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
df = df.drop(columns=['geometry', 'latitude', 'longitude'])

#add point to neighbourhoods
# Split the geoLocation column into latitude and longitude columns
df[['latitude', 'longitude']] = df['input_neighbourhood_geoLocation'].str.split(',', expand=True)

# Format the data into the expected SQL format
df['neighbourhoodInput'] = df.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
df = df.drop(columns=['input_neighbourhood_geoLocation', 'latitude', 'longitude'])

df = df.rename(columns = {"neighbourhoodInput" : "neighbourhoodGeoLocation"})

In [ ]:
df.to_csv("D:/DataSets/NewServices/supermarkets.csv", index = False)

# FILTERED TRAIN STATION

In [89]:
filtered_train_station = pd.read_csv("D:/DataSets/Services/Filtered_train_station.csv")
df = filtered_train_station.copy()


In [ ]:
# Create a list to store all nightclubs
all_businesses_clubs = []

for index, row in neighborhood_df.iterrows():
    input_neighbourhood = row["neighbourhoodGeoLocation"]
    neighborhood_businesses = fetch_businesses(input_neighbourhood,"train_station")
    all_businesses_clubs.extend(neighborhood_businesses)

# Create a DataFrame from the combined list of nightclubs
df = pd.DataFrame(all_businesses_clubs)

df['geometry'] = df['geometry'].astype(str)

# fix the geometry string
for i in range(len(df)):
    input_str = df["geometry"][i]
    data_dict = ast.literal_eval(input_str)
    lat = data_dict['location']['lat']
    lng = data_dict['location']['lng']
    result_str = f"{lat} {lng}"
    df["geometry"][i] = result_str
    
#add point to geometry
# Split the geoLocation column into latitude and longitude columns
df[['latitude', 'longitude']] = df['geometry'].str.split(' ', expand=True)

# Format the data into the expected SQL format
df['geoLocation'] = df.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
df = df.drop(columns=['geometry', 'latitude', 'longitude'])

#add point to neighbourhoods
# Split the geoLocation column into latitude and longitude columns
df[['latitude', 'longitude']] = df['input_neighbourhood_geoLocation'].str.split(',', expand=True)

# Format the data into the expected SQL format
df['neighbourhoodInput'] = df.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
df = df.drop(columns=['input_neighbourhood_geoLocation', 'latitude', 'longitude'])

df = df.rename(columns = {"neighbourhoodInput" : "neighbourhoodGeoLocation"})

In [ ]:
df.to_csv("D:/DataSets/NewServices/train_stations.csv", index = False)

# FILTERED TRANSIT STATION

In [90]:
filtered_transit = pd.read_csv("D:/DataSets/Services/Filtered_transit_station.csv")
df = filtered_transit.copy()


In [ ]:
# Create a list to store all nightclubs
all_businesses_clubs = []

for index, row in neighborhood_df.iterrows():
    input_neighbourhood = row["neighbourhoodGeoLocation"]
    neighborhood_businesses = fetch_businesses(input_neighbourhood,"transit_station")
    all_businesses_clubs.extend(neighborhood_businesses)

# Create a DataFrame from the combined list of nightclubs
df = pd.DataFrame(all_businesses_clubs)

df['geometry'] = df['geometry'].astype(str)

# fix the geometry string
for i in range(len(df)):
    input_str = df["geometry"][i]
    data_dict = ast.literal_eval(input_str)
    lat = data_dict['location']['lat']
    lng = data_dict['location']['lng']
    result_str = f"{lat} {lng}"
    df["geometry"][i] = result_str
    
#add point to geometry
# Split the geoLocation column into latitude and longitude columns
df[['latitude', 'longitude']] = df['geometry'].str.split(' ', expand=True)

# Format the data into the expected SQL format
df['geoLocation'] = df.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
df = df.drop(columns=['geometry', 'latitude', 'longitude'])

#add point to neighbourhoods
# Split the geoLocation column into latitude and longitude columns
df[['latitude', 'longitude']] = df['input_neighbourhood_geoLocation'].str.split(',', expand=True)

# Format the data into the expected SQL format
df['neighbourhoodInput'] = df.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
df = df.drop(columns=['input_neighbourhood_geoLocation', 'latitude', 'longitude'])

df = df.rename(columns = {"neighbourhoodInput" : "neighbourhoodGeoLocation"})

In [ ]:
df.to_csv("D:/DataSets/NewServices/transit_stations.csv", index = False)